# **Brain Tumor Segmentation**

## **Libraries**

In [ ]:
import os
import keras
#!pip install nibabel
import nibabel as nib
from PIL import Image
#!pip install sklearn
from sklearn.utils import class_weight
import tensorflow as tf

import os
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
# from medpy.io import load
import numpy as np

#import cv2
import nibabel as nib
from PIL import Image
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## **Segmentation model**

In [ ]:

def standardize(image):

  standardized_image = np.zeros(image.shape)

  #
  
      # iterate over the `z` dimension
  for z in range(image.shape[2]):
      # get a slice of the image 
      # at channel c and z-th dimension `z`
      image_slice = image[:,:,z]

      # subtract the mean from image_slice
      centered = image_slice - np.mean(image_slice)
      
      # divide by the standard deviation (only if it is different from zero)
      if(np.std(centered)!=0):
          centered = centered/np.std(centered) 

      # update  the slice of standardized image
      # with the scaled centered and scaled image
      standardized_image[:, :, z] = centered

  ### END CODE HERE ###

  return standardized_image


def dice_coef(y_true, y_pred, epsilon=0.00001):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    
    """
    axis = (0,1,2,3)
    dice_numerator = 2. * K.sum(y_true * y_pred, axis=axis) + epsilon
    dice_denominator = K.sum(y_true*y_true, axis=axis) + K.sum(y_pred*y_pred, axis=axis) + epsilon
    return K.mean((dice_numerator)/(dice_denominator))

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def DilatedConvolution3DBlock(input_mat,num_filters,kernel_size, channels, g, stride, batch_norm, dilation ):
  X = tf.keras.layers.BatchNormalization()(input_mat)
  X = tf.keras.layers.Conv3D(num_filters,kernel_size=(kernel_size,kernel_size,kernel_size),strides=(1,1,1),padding='same')(X)
  X = tf.keras.layers.Activation('relu')(X)
  return X

def DMFUnit(input_mat,num_filters,kernel_size, channels, g, stride, batch_norm, dilation ):
  X1 = DilatedConvolution3DBlock(input_mat,num_filters,kernel_size, channels, g, stride, batch_norm, dilation )
  X2 = DilatedConvolution3DBlock(X1,num_filters,kernel_size, channels, g, stride, batch_norm, dilation )
  X3 = DilatedConvolution3DBlock(X2,num_filters,kernel_size, channels, g, stride, batch_norm, dilation )
  X = tf.keras.layers.concatenate([X1,X2,X3]);
  print('X1',X1.shape)
  print('X2',X2.shape)
  print('X3',X3.shape)
  print('X',X.shape)
  X = DilatedConvolution3DBlock(X,num_filters,kernel_size, channels, g, stride, batch_norm, dilation )
  print('X',X.shape)
  return X
def DMFUnit(input_mat,n_filters,kernel, channels, g, strides, batch_norm, dilation ):
    batch_normalization=True
    activation=False
    print('input_layer',input_mat.shape)
    layer1 = tf.keras.layers.Conv3D(n_filters, kernel, padding='same', strides=strides,dilation_rate=1)(input_mat)
    print('layer1',layer1.shape)
    if batch_normalization:
        layer1 = tf.keras.layers.BatchNormalization(axis=1)(layer1)
    elif instance_normalization:
        # import tensorflow_addons as tfa
        layer1 = tf.keras.layers.BatchNormalization(axis=1)(layer1)
    if activation is None:
        layer1= tf.keras.layers.Activation('relu')(layer1)
    else:
        layer1= activation()(layer1)
    ######################### 
    layer2 = tf.keras.layers.Conv3D(n_filters, kernel, padding='same', strides=strides,dilation_rate=2)(input_layer)
    if batch_normalization:
        layer2 = tf.keras.layers.BatchNormalization(axis=1)(layer2)
    elif instance_normalization:
        # import tensorflow_addons as tfa
        layer2 = tf.keras.layers.BatchNormalization(axis=1)(layer2)
    if activation is None:
        layer2= tf.keras.layers.Activation('relu')(layer2)
    else:
        layer2= activation()(layer2)
    ######################### 
    layer3 = tf.keras.layers.Conv3D(n_filters, kernel, padding='same', strides=strides,dilation_rate=3)(input_layer)
    if batch_normalization:
        layer3 = tf.keras.layers.BatchNormalization(axis=1)(layer3)
    elif instance_normalization:
        # import tensorflow_addons as tfa
        layer3 = tf.keras.layers.BatchNormalization(axis=1)(layer3)
    if activation is None:
        layer3= tf.keras.layers.Activation('relu')(layer3)
    else:
        layer3= activation()(layer3)
    layer=tf.keras.layers.concatenate([layer1,layer2,layer3]);
    print('layer1',layer1.shape)
    print('layer2',layer2.shape)
    print('layer3',layer3.shape)
    print('layer',layer.shape)
    layer = tf.keras.layers.Conv3D(n_filters, kernel, padding='same', strides=strides,dilation_rate=1)(layer)
    print('layer',layer.shape)
    return layer
c=4,
n=32,
channels=128, 
groups=16,
norm='bn',
num_classes=4

def MLDC(input_img, n_filters = 8, dropout = 0.2, batch_norm = True):
   print('Input Image',input_img.shape)
   c1=DMFUnit(input_img,32,3, channels, g=groups, strides=2, batch_norm=True,dilation=[1,2,3])
   print('C1',c1.shape)
   
   p1 = tf.keras.layers.MaxPooling3D(pool_size=(2,2,2) ,strides=2,padding='valid')(c1)
   print('p1',p1.shape)
   c2=DMFUnit(p1,n_filters*2, 3, channels, g=groups, stride=2, batch_norm=True,dilation=[1,2,3]) 
   print('C2',c2.shape)

   p2 = tf.keras.layers.MaxPooling3D(pool_size=(2,2,2) ,strides=2,padding='valid')(c2)
   print('p2',p2.shape)
   
   c3=DMFUnit(p2,n_filters*4,3, channels, g=groups, stride=2, batch_norm=True,dilation=[1,2,3])
   print('c3',c3.shape)
  
   u1 = tf.keras.layers.Conv3DTranspose(n_filters*4,(3,3,3),strides = (2,2,2) , padding='same')(c3)
   print('u1',u1.shape)
   con1 = tf.keras.layers.concatenate([u1,c2]);
   print('con1',con1.shape)
   c4 = DMFUnit(con1,n_filters*2, 3, channels, g=groups, stride=2, batch_norm=True,dilation=[1,2,3]) 
   print('c4',c4.shape)
    
   u2 = tf.keras.layers.Conv3DTranspose(n_filters*4,(3,3,3),strides = (2,2,2) , padding='same')(c4)
   print('u2',u2.shape)
   con2 = tf.keras.layers.concatenate([u2,c1]);
   print('con2',con2.shape)
   c5 = DMFUnit(con2,n_filters*2, 3, channels, g=groups, stride=2, batch_norm=True,dilation=[1,2,3]) 
   print('c5',c5.shape)
   outputs = tf.keras.layers.Conv3D(4, (1, 1,1), activation='softmax')(c5)
   print('outputs',outputs.shape)
  #  print(outputs.shape)
   model = tf.keras.Model(inputs=input_img, outputs=outputs)

   return model

 
input_img = tf.keras.Input((128,128,128,4))
model = MLDC(input_img,8,0.1,True)
learning_rate = 0.001
epochs = 5000
decay_rate = 0.0000001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate, decay = decay_rate), loss=dice_coef_loss, metrics=[dice_coef,'acc'])
model.summary()



## **Preprocessing**

In [ ]:


path = '/content/drive/MyDrive/Colab Notebooks/ASmall_dataset'
all_images = os.listdir(path)
#print(len(all_images))
all_images.sort()
data = np.zeros((240,240,155,4))
image_data2=np.zeros((240,240,155))
train_data=np.zeros((5,128,128,128,4))
labels=np.zeros((5,128,128,128,4))

for image_num in range(369):
    train_label=np.zeros((128,128,128))
# data preprocessing starts here
    x = all_images[image_num]
    print(x)
    folder_path = path + '/' + x;
    modalities = os.listdir(folder_path)
    modalities.sort()
    #data = []
    w = 0
    for j in range(len(modalities)):
      #print(modalities[j])
      
      image_path = folder_path + '/' + modalities[j]
      if(image_path[-10:-1] + image_path[-1] == 'seg.nii.gz'):
        img = nib.load(image_path);
        image_data2 = img.get_data()
        image_data2 = np.asarray(image_data2)
        print("Entered ground truth")
      else:
        img = nib.load(image_path);
        image_data = img.get_data()
        image_data = np.asarray(image_data)
        image_data = standardize(image_data)
        data[:,:,:,w] = image_data
        print("Entered modality",w, image_path[-10:-1] + image_path[-1] )
        w = w+1
      
    print('4 modalities shape',data.shape)
    print('label shape',image_data2.shape)  

    reshaped_data=data[56:184,80:208,13:141,:]
    reshaped_image_data2=image_data2[56:184,80:208,13:141]
    print('Cropped 4 modalities shape',reshaped_data.shape)
    print('Cropped label shape',reshaped_image_data2.shape)  

    
    reshaped_data=reshaped_data.reshape(128,128,128,4)
    print('reshaped modality', reshaped_data.shape)
    train_data[image_num,:,:,:,:] = reshaped_data
    # train_data.append(reshaped_data)

    reshaped_image_data2=reshaped_image_data2.reshape(128,128,128)
    train_label[:,:,:] = reshaped_image_data2
    
    print('reshaped label ', reshaped_image_data2.shape)
    train_label[train_label==4] = 3
    print('label4=3',train_label.shape)
    hello = train_label.flatten()
    print('hello',train_label.shape)

        #y_to = keras.utils.to_categorical(y_to,num_classes=2)
    print('hello[hello==3]',hello[hello==3].shape)
    print("Number of classes",np.unique(hello))

    class_weights = class_weight.compute_class_weight('balanced',np.unique(hello),hello)
    print('class weights',class_weights)
    train_label = tf.keras.utils.to_categorical(train_label, num_classes = 4)
    print('labe to categorical',train_label.shape)
    print(type(reshaped_data))
    # train_label.append(reshaped_image_data2)
    labels[image_num,:,:,:,:]=train_label
train_data=np.array(train_data)
labels=np.array(labels)
print(train_data.shape)
print(labels.shape)
plt.imshow(train_data[1,:,:,80,2])


## **Training**

In [ ]:

model.fit(train_data, labels, batch_size=2, epochs=500)
fig, ax = plt.subplots(1, 2, figsize=(20, 5))
ax = ax.ravel()

for i, metric in enumerate(["acc", "loss"]):
    ax[i].plot(model.history.history[metric])
    ax[i].plot(model.history.history["val_" + metric])
    ax[i].set_title("Model {}".format(metric))
    ax[i].set_xlabel("epochs")
    ax[i].set_ylabel(metric)
    ax[i].legend(["train", "val"])

## **Testing**

In [ ]:

import os

path = '/content/drive/MyDrive/MICCAI_BraTS2020_ValidationData/'

all_images = os.listdir(path)
print(len(all_images))
all_images.sort()
instances=125
data = np.zeros((240,240,155,4))
chunks_merged=np.zeros((240,240,155))

val_data=np.zeros((instances,128,128,128,4))
# brats19_prediction='D:/kiran/brats2019/prediction2_basicunet_4/'
chunk1= np.zeros((128,128,128,4))
chunk2= np.zeros((128,128,128,4))
chunk3= np.zeros((128,128,128,4))
chunk4= np.zeros((128,128,128,4))


for image_num in range(instances):
    chunks=[]
    print(image_num)
    #train_label=np.zeros((128,128,128))
    x = all_images[image_num]
    
    vol_name=x
    print('patient',vol_name)
    folder_path = path + '/' + x;
    modalities = os.listdir(folder_path)
    print(modalities)
    modalities.sort()
    #data = []
    w = 0
    for j in range(len(modalities)):
      print(modalities[j])
      image_path = folder_path + '/' + modalities[j]
      print(image_path)
      img = nib.load(image_path);
      image_data = img.get_data()
      image_data = np.asarray(image_data)
      image_data = standardize(image_data)
      data[:,:,:,w] = image_data
       #print("Entered modality",w, image_path[-10:-1] + image_path[-1] )
      w = w+1
    
    
    print('4 modalities shape',data.shape) 
   
    #divide 240*240*155 sized image to 4 128*128*128 images
    chunk1[8:128,8:128,:,:]=data[0:120,0:120,13:141,:] 
    reshaped_chunk1=chunk1.reshape(128,128,128,4)
    plt.imshow(reshaped_chunk1[:,:,80,0])
    plt.show()
    
    chunk2[0:120,8:128,:,:]=data[120:240,0:120,13:141,:] 
    reshaped_chunk2=chunk2.reshape(128,128,128,4)
    plt.imshow(reshaped_chunk2[:,:,80,0])
    plt.show()
    
    chunk3[8:128,0:120,:,:]=data[0:120,120:240,13:141,:] 
    reshaped_chunk3=chunk3.reshape(128,128,128,4)
    plt.imshow(reshaped_chunk3[:,:,80,0])
    plt.show()
    
    chunk4[0:120,0:120,:,:]=data[120:240,120:240,13:141,:]
    reshaped_chunk4=chunk4.reshape(128,128,128,4)
    plt.imshow(reshaped_chunk4[:,:,80,0])
    plt.show()
   
    #predicting sub patch 1 and adding it back to its place in dummy array
    reshaped_chunk1=reshaped_chunk1.reshape(1,128,128,128,4)
    prediction=model.predict(reshaped_chunk1)
    _classes=prediction.argmax(axis=-1)
    print('classes',_classes.shape)
    chunks_merged[0:120,0:120,13:141]=_classes[0,8:128,8:128,:]

    #predicting sub patch 2 and adding it back to its place in dummy array
    reshaped_chunk2=reshaped_chunk2.reshape(1,128,128,128,4)
    prediction=model.predict(reshaped_chunk2)
    _classes=prediction.argmax(axis=-1)
    print('classes',_classes.shape)
    chunks_merged[120:240,0:120,13:141]=_classes[0,0:120,8:128,:]

    #predicting sub patch 3 and adding it back to its place in dummy array
    reshaped_chunk3=reshaped_chunk3.reshape(1,128,128,128,4)
    prediction=model.predict(reshaped_chunk3)
    _classes=prediction.argmax(axis=-1)
    print('classes',_classes.shape)
    chunks_merged[0:120,120:240,13:141]=_classes[0,8:128,0:120,:]

    #predicting sub patch 4 and adding it back to its place in dummy array
    reshaped_chunk4=reshaped_chunk4.reshape(1,128,128,128,4)
    prediction=model.predict(reshaped_chunk4)
    _classes=prediction.argmax(axis=-1)
    print('classes',_classes.shape)
    chunks_merged[120:240,120:240,13:141]=_classes[0,0:120,0:120,:]

    chunks_merged=chunks_merged[0:240,0:240,:]
    #chunks_merged=np.swapaxes(chunks_merged,0,1)
    chunks_merged[chunks_merged==3]=4
    chunks_merged=chunks_merged.astype('uint8')
    plt.imshow(chunks_merged[:,:,80])
    plt.show()
    # nib.save(nib.Nifti1Image(chunks_merged, None),brats19_prediction+vol_name+'.nii.gz')

chunks_merged.shape


# **Overall Survival-time prediction**

In [ ]:
!pip install SimpleITK
!pip install pyradiomics 
import os
import numpy
import six
import pandas as pd
import glob
import nibabel as nib
import numpy as np
from glob import glob
# from keras.utils import multi_gpu_model
from keras import optimizers
import matplotlib.pyplot as plt
import collections as clt
import SimpleITK as sitk
import radiomics
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
from radiomics import featureextractor, getTestCase, getFeatureClasses
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm, getTestCase,gldm,ngtdm
import scipy.stats 

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error 

## **Setting**

In [ ]:
applyLog = True
applyWavelet = True

settings = {'binWidth': 25,
            'interpolator': sitk.sitkBSpline,
            'resampledPixelSpacing': None}


## **Feature Extraction**

In [ ]:
#Training
# path_modalities='/content/drive/MyDrive/Colab Notebooks/brats2019/MICCAI_BraTS_2019_Data_Training/HGG'
# path_HGG_2019 = glob(path_modalities+'/**')
# path_predicted='/content/drive/MyDrive/Colab Notebooks/Survival_prediction_Atrous/357/TrainDataOutput/submission'
# path_train_468 = glob(path_predicted+'/**')

#Validation
path_modalities='/content/drive/MyDrive/Colab Notebooks/brats2019/MICCAI_BraTS_2019_Data_Validation'
path_HGG_2019 = glob(path_modalities+'/**')
path_predicted='/content/drive/MyDrive/Colab Notebooks/Survival_prediction_Atrous/123/TestDataOutput/submission'
path_train_468 = glob(path_predicted+'/**')
# mod=''

for view in ['axial_','coronal_','sagittal_']:
  for modality in ['_t1ce','_t1','_t2','_flair']:
    for i in range(len(df['BraTS19ID'])):
          print('i',i)
          for s in path_HGG_2019:
            # print('s',s)
            if df['BraTS19ID'][i] in s:
              print('@@@@@@@@@@@@@@@@@@@patient',i)
              if i != 66:
                print('s',s)
                if view=='axial_':
                  axis1=1
                  axis2=2
                  mod='axial'+modality+'_'
                if view=='coronal_':
                  axis1=0
                  axis2=1
                  mod='coronal'+modality+'_'
                if view=='sagittal_':
                  axis1=0
                  axis2=2
                  mod='sagittal'+modality+'_'
                image=readData(os.path.join(s+'/'+df['BraTS19ID'][i]+modality+'.nii.gz'),axis1,axis2)
                mask=readData(os.path.join(path_predicted+'/'+df['BraTS19ID'][i]+'.nii.gz'),axis1,axis2)
                print('image',os.path.join(s+'/'+df['BraTS19ID'][i]+modality+'.nii.gz'))
                print('mask', os.path.join(path_predicted+'/'+df['BraTS19ID'][i]+'.nii.gz'))
                print('view',view)
                print('mod',mod)
                print()
                #
                # If enabled, resample image (resampled image is automatically cropped.
                #
                interpolator = settings.get('interpolator')
                resampledPixelSpacing = settings.get('resampledPixelSpacing')
                if interpolator is not None and resampledPixelSpacing is not None:
                  image, mask = imageoperations.resampleImage(image, mask, **settings)

                bb, correctedMask = imageoperations.checkMask(image, mask)
                if correctedMask is not None:
                  mask = correctedMask
                image, mask = imageoperations.cropToTumorMask(image, mask, bb)
                
                # #################################################################################################################################################
                # Show Shape features
                shapeFeatures = shape.RadiomicsShape(image, mask, **settings)
                shapeFeatures.enableAllFeatures()
                #print('Calculating Shape features...')
                results = shapeFeatures.execute()
                for (key, val) in six.iteritems(results):
                  key=mod+key
                  # print('i=',i,' ', key, ':', val)
                  df[key][i]=val 

                firstOrderFeatures = firstorder.RadiomicsFirstOrder(image, mask, **settings)
                firstOrderFeatures.enableAllFeatures()
                results = firstOrderFeatures.execute()
                for (key, val) in six.iteritems(results):
                  key=mod+key
                  #print('i=',i,'  ', key, ':', val)
                  df[key][i]=val 
                  
                
                # Show GLCM features
                glcmFeatures = glcm.RadiomicsGLCM(image, mask, **settings)
                glcmFeatures.enableAllFeatures()
                #print('Calculating GLCM features...')
                results = glcmFeatures.execute()
                for (key, val) in six.iteritems(results):
                  key=mod+key
                  #print('  ', key, ':', val)
                  df[key][i]=val

                
                # Show GLRLM features
                glrlmFeatures = glrlm.RadiomicsGLRLM(image, mask, **settings)
                glrlmFeatures.enableAllFeatures()
                #print('Calculating GLRLM features...')
                results = glrlmFeatures.execute()
                for (key, val) in six.iteritems(results):
                  key=mod+key
                  #print('  ', key, ':', val)
                  df[key][i]=val

                
                # Show GLSZM features
                glszmFeatures = glszm.RadiomicsGLSZM(image, mask, **settings)
                glszmFeatures.enableAllFeatures()
                #print('Calculating GLSZM features...')
                results = glszmFeatures.execute()
                for (key, val) in six.iteritems(results):
                  key=mod+key
                  #print('  ', key, ':', val)
                  df[key][i]=val
                
                # Show GLDM features
                gldmFeatures = gldm.RadiomicsGLDM(image, mask, **settings)
                gldmFeatures.enableAllFeatures()
                #print('Calculating GLDM features...')
                results = gldmFeatures.execute()
                for (key, val) in six.iteritems(results):
                  key=mod+key
                  #print('  ', key, ':', val)
                  df[key][i]=val 

                # Show NGTDM features
                ngtdmFeatures = ngtdm.RadiomicsNGTDM(image, mask, **settings)
                ngtdmFeatures.enableAllFeatures()
                results = ngtdmFeatures.execute()
                #print('Calculated NGTDM features ')
                for (key, val) in six.iteritems(results):
                    key=mod+key
                    #print('  ', key, ':', val)  
                    df[key][i]=val
                #################################################################################################################################################
                #
                # 
                #
                if applyLog:
                  sigmaValues = numpy.arange(5., 0., -.5)[::1]
                  for logImage, imageTypeName, inputKwargs in imageoperations.getLoGImage(image, mask, sigma=sigmaValues):
                    
                    #Show FirstOrder features, calculated on a LoG filtered image
                    logFirstorderFeatures = firstorder.RadiomicsFirstOrder(logImage, mask, **inputKwargs)
                    logFirstorderFeatures.enableAllFeatures()
                    results = logFirstorderFeatures.execute()
                    for (key, val) in six.iteritems(results):
                      laplacianFeatureName = '%s_%s' % (imageTypeName, key)
                      laplacianFeatureName=mod+laplacianFeatureName
                      #print('  ', laplacianFeatureName, ':', val)
                      df[laplacianFeatureName][i]=val
                    
                    
                    # Show GLCM  features, calculated on a LoG filtered image
                    logglcmFeatures = glcm.RadiomicsGLCM(logImage, mask, **inputKwargs)
                    logglcmFeatures.enableAllFeatures()
                    results = logglcmFeatures.execute()
                    for (key, val) in six.iteritems(results):
                      laplacianFeatureName = '%s_%s' % (imageTypeName, key)
                      laplacianFeatureName=mod+laplacianFeatureName
                      #print('  ', laplacianFeatureName, ':', val)
                      df[laplacianFeatureName][i]=val

                    
                    # Show GLRLM features
                    logglrlmFeatures = glrlm.RadiomicsGLRLM(logImage, mask, **inputKwargs)
                    logglrlmFeatures.enableAllFeatures()
                    #print('Calculating GLRLM features...')
                    results = logglrlmFeatures.execute()
                    for (key, val) in six.iteritems(results):
                      laplacianFeatureName = '%s_%s' % (imageTypeName, key)
                      laplacianFeatureName=mod+laplacianFeatureName
                      #print('  ', laplacianFeatureName, ':', val)
                      df[laplacianFeatureName][i]=val

                    
                    # Show GLSZM features
                    logglszmFeatures = glszm.RadiomicsGLSZM(logImage, mask, **inputKwargs)
                    logglszmFeatures.enableAllFeatures()
                    #print('Calculating GLSZM features...')
                    results = logglszmFeatures.execute()
                    for (key, val) in six.iteritems(results):
                      laplacianFeatureName = '%s_%s' % (imageTypeName, key)
                      laplacianFeatureName=mod+laplacianFeatureName
                      #print('  ', laplacianFeatureName, ':', val)
                      df[laplacianFeatureName][i]=val
                    
                    # Show GLDM features
                    loggldmFeatures = gldm.RadiomicsGLDM(logImage, mask, **inputKwargs)
                    loggldmFeatures.enableAllFeatures()
                    #print('Calculating GLDM features...')
                    results = loggldmFeatures.execute()
                    for (key, val) in six.iteritems(results):
                      laplacianFeatureName = '%s_%s' % (imageTypeName, key)
                      laplacianFeatureName=mod+laplacianFeatureName
                      #print('  ', laplacianFeatureName, ':', val) 
                      df[laplacianFeatureName][i]=val

                    # Show NGTDM features
                    logngtdmFeatures = ngtdm.RadiomicsNGTDM(logImage, mask, **inputKwargs)
                    logngtdmFeatures.enableAllFeatures()
                    results = logngtdmFeatures.execute()
                    #print('Calculated NGTDM features ')
                    for (key, val) in six.iteritems(results):
                      laplacianFeatureName = '%s_%s' % (imageTypeName, key)
                      laplacianFeatureName=mod+laplacianFeatureName
                      #print('  ', laplacianFeatureName, ':', val)  
                      df[laplacianFeatureName][i]=val


                #################################################################################################################################################
                # Show FirstOrder features, calculated on a wavelet filtered image
                #
                if applyWavelet:
                  for decompositionImage, decompositionName, inputKwargs in imageoperations.getWaveletImage(image, mask):
                    waveletFirstOrderFeaturs = firstorder.RadiomicsFirstOrder(decompositionImage, mask, **inputKwargs)
                    waveletFirstOrderFeaturs.enableAllFeatures()
                    results = waveletFirstOrderFeaturs.execute()
                    #print('Calculated firstorder features with wavelet ', decompositionName)
                    for (key, val) in six.iteritems(results):
                      waveletFeatureName = '%s_%s' % (str(decompositionName), key)
                      #print(key)
                      waveletFeatureName=mod+waveletFeatureName
                      #print('  ', waveletFeatureName, ':', val)
                      df[waveletFeatureName][i]=val 
                  
                  for decompositionImage, decompositionName, inputKwargs in imageoperations.getWaveletImage(image, mask):
                    waveletglszmFeatures = glszm.RadiomicsGLSZM(decompositionImage, mask, **inputKwargs)
                    waveletglszmFeatures.enableAllFeatures()
                    results = waveletglszmFeatures.execute()
                    #print('Calculated glszm features with wavelet ', decompositionName)
                    for (key, val) in six.iteritems(results):
                      waveletFeatureName = '%s_%s' % (str(decompositionName), key)
                      #print(key)
                      waveletFeatureName=mod+waveletFeatureName
                      #print('  ', waveletFeatureName, ':', val)
                      df[waveletFeatureName][i]=val 

                for decompositionImage, decompositionName, inputKwargs in imageoperations.getWaveletImage(image, mask):
                    waveletglrlmFeatures = glrlm.RadiomicsGLRLM(decompositionImage, mask, **inputKwargs)
                    waveletglrlmFeatures.enableAllFeatures()
                    results = waveletglrlmFeatures.execute()
                    #print('Calculated GLRLM features with wavelet ', decompositionName)
                    for (key, val) in six.iteritems(results):
                      waveletFeatureName = '%s_%s' % (str(decompositionName), key)
                      #print(key)
                      waveletFeatureName=mod+waveletFeatureName
                      #print('  ', waveletFeatureName, ':', val)
                      df[waveletFeatureName][i]=val 
                for decompositionImage, decompositionName, inputKwargs in imageoperations.getWaveletImage(image, mask):
                    waveletglcmFeatures = glcm.RadiomicsGLCM(decompositionImage, mask, **inputKwargs)
                    waveletglcmFeatures.enableAllFeatures()
                    results = waveletglcmFeatures.execute()
                    #print('Calculated GLCM features with wavelet ', decompositionName)
                    for (key, val) in six.iteritems(results):
                      waveletFeatureName = '%s_%s' % (str(decompositionName), key)
                      #print(key)
                      waveletFeatureName=mod+waveletFeatureName
                      #print('  ', waveletFeatureName, ':', val)
                      df[waveletFeatureName][i]=val 
                for decompositionImage, decompositionName, inputKwargs in imageoperations.getWaveletImage(image, mask):
                      waveletgldmFeatures = gldm.RadiomicsGLDM(decompositionImage, mask, **inputKwargs)
                      waveletgldmFeatures.enableAllFeatures()
                      results = waveletgldmFeatures.execute()
                      #print('Calculated GLDM features with wavelet ', decompositionName)
                      for (key, val) in six.iteritems(results):
                        waveletFeatureName = '%s_%s' % (str(decompositionName), key)
                        waveletFeatureName=mod+waveletFeatureName
                        #print('  ', waveletFeatureName, ':', val)
                        df[waveletFeatureName][i]=val 
                for decompositionImage, decompositionName, inputKwargs in imageoperations.getWaveletImage(image, mask):
                      waveletngtdmFeatures = ngtdm.RadiomicsNGTDM(decompositionImage, mask, **inputKwargs)
                      waveletngtdmFeatures.enableAllFeatures()
                      results = waveletngtdmFeatures.execute()
                      #print('Calculated NGTDM features with wavelet ', decompositionName)
                      for (key, val) in six.iteritems(results):
                        waveletFeatureName = '%s_%s' % (str(decompositionName), key)
                        waveletFeatureName=mod+waveletFeatureName
                        #print('  ', waveletFeatureName, ':', val)
                        df[waveletFeatureName][i]=val  

# 123
# 468
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Survival_prediction_Atrous/123/validation_all_features_123.csv')

## **preprocessing**

In [ ]:
count=len(df_train['BraTS19ID'])
df_train["label"] = ""
for i in range(count): 
  if (i!=66): 
    if(df_train['Survival'][i]>0 and df_train['Survival'][i]<300):
      df_train['label'][i]='low'
    if(df_train['Survival'][i]>=300 and df_train['Survival'][i]<450):
      df_train['label'][i]='medium'
    if(df_train['Survival'][i]>=450 ):
      df_train['label'][i]='high'
df_train

In [ ]:
df_train=df_train.drop([66])
df_train=df_train.fillna(0)
df_train=df_train.drop(df_train.columns[0],axis=1)
df_train=df_train.drop(df_train.columns[0],axis=1)
lbl=df_train['label']
train_label=df_train['Survival']
df_train= df_train.drop(['BraTS19ID','Survival','label'], axis = 1) 
df_train


df_val=df_val.fillna(0)
df_val=df_val.drop(df_val.columns[0],axis=1) 
df_val=df_val.drop(df_val.columns[0],axis=1) 
val_patients=df_val['BraTS19ID']
df_val= df_val.drop(['BraTS19ID'], axis = 1) 
df_val.head()

## **Feature Selection**

In [ ]:
import pandas as pd
import numpy as np

X = ff  #independent columns
y = lbl  #target column i.e price range
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,y)
#print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
print(feat_importances.nlargest(7))
feat_importances.nlargest(7).plot(kind='barh')
plt.show()


## **Training and testing**

In [ ]:
features=['axial_t2_log-sigma-3-5-mm-3D_Skewness', 'axial_flair_log-sigma-5-0-mm-3D_SmallAreaHighGrayLevelEmphasis', 'axial_flair_wavelet-LHH_LowGrayLevelZoneEmphasis', 'axial_t2_log-sigma-1-0-mm-3D_SumSquares', 'axial_flair_wavelet-HLH_ShortRunHighGrayLevelEmphasis', 'axial_flair_log-sigma-3-0-mm-3D_Minimum', 'Age'] 
X_train=df_train[features]
X_test=df_val[features]
# X_train.head()
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
regressor=RandomForestRegressor(n_estimators=200,random_state=69)

regressor.fit(X_train, train_label)
predictions_train = regressor.predict(X_train)
predictions_val = regressor.predict(X_test)

# pred =  pd.DataFrame(predictions_train)
# pred['BraTS19ID']=train_p_id
pred2 =  pd.DataFrame(predictions_val)
pred2['BraTS19ID']=val_patients

# pred2.to_csv('/content/drive/MyDrive/Colab Notebooks/Survival_prediction_Atrous/246/Survival_prediction/Axial_Coronal/ax_co_246_val_6.csv')
pred2
